In [1]:
import numpy as np
import pandas as pd
import json
import urllib.request as urlreq
import os
import sys

In [7]:
!curl -O https://raw.githubusercontent.com/RWD2E/phecdm/refs/heads/main/src/json_util.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17264  100 17264    0     0  66400      0 --:--:-- --:--:-- --:--:-- 66145


In [3]:
print(sys.path)
print(os.getcwd())

# add path to current project dir
sys.path.append(os.getcwd()) # run this line only if the module can't be found 
unique_syspath = list(dict.fromkeys(sys.path))
sys.path = unique_syspath
print(sys.path)

# import downloaded python function
from json_util import QueryFromJson

['/tmp/spark-32e7e652-b678-4567-aa07-3f793b12f99b/userFiles-c71f619e-07e0-44fc-aeb4-ed4b0fc88ccf', '/home/mu_research2/work/University of Missouri/DRIVERS', '/usr/local/spark/python', '/usr/local/spark/python/lib/py4j-0.10.7-src.zip', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/home/mu_research2/.ipython']
/home/mu_research2/work/University of Missouri/DRIVERS
['/tmp/spark-32e7e652-b678-4567-aa07-3f793b12f99b/userFiles-c71f619e-07e0-44fc-aeb4-ed4b0fc88ccf', '/home/mu_research2/work/University of Missouri/DRIVERS', '/usr/local/spark/python', '/usr/local/spark/python/lib/py4j-0.10.7-src.zip', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/home/mu_research2/.ipython']


In [4]:
path_to_data = os.path.join(os.getcwd(),'data')
path_to_data

'/home/mu_research2/work/University of Missouri/DRIVERS/data'

In [5]:
spark.sql("use real_world_data_ed_omop_nov_2024")

DataFrame[]

In [9]:
# load delivery code list and save as a temp view
vs_delivery = QueryFromJson(
    url = 'https://raw.githubusercontent.com/RWD2E/phecdm/refs/heads/main/res/valueset_curated/vs-mmm-cde.json',
    sqlty = 'spark',
    cd_field = 'CONCEPT_CODE',
    cdtype_field = 'VOCABULARY_ID',
    srctbl_name = "CONCEPT",
    other_fields=["concept_id","concept_name","domain_id"],
    sel_keys = ['vaginalDelivery','cSection']
)
vs_delivery_cd_qry = vs_delivery.gen_qry()
print(vs_delivery_cd_qry)

Enter Code Type Value for icd9cm: ICD9CM
Enter Code Type Value for icd10cm: ICD10CM
Enter Code Type Value for icd9proc: ICD9PROC
Enter Code Type Value for icd10pcs: ICD10PCS
Enter Code Type Value for cpt: CPT4
Enter Code Type Value for hpc: HCPCS
Enter Code Type Value for drg: MSDRG
Enter Code Type Value for loinc: LOINC

                    select concept_id,concept_name,domain_id,CONCEPT_CODE,VOCABULARY_ID ,'vaginalDelivery' as CD_GRP
                    from CONCEPT
                    where (VOCABULARY_ID='ICD9CM' and (
                        substring(CONCEPT_CODE,1,6) in ('669.70')
                    )) OR (VOCABULARY_ID='ICD10CM' and (
            substring_index(CONCEPT_CODE,'.',1)
         in ('O80')
                        )) OR (VOCABULARY_ID='MSDRG' and (
                        CONCEPT_CODE in ('767','768','774','775','796','797','798','85','86','87')
                    )) OR (VOCABULARY_ID='CPT4' and (
                        CONCEPT_CODE in ('59400','59401','59402','5

In [10]:
delivery_omop_meta = spark.sql(vs_delivery_cd_qry).cache()
delivery_omop_meta.createOrReplaceTempView("delivery_omop_meta")
delivery_omop_meta.first()

Row(concept_id=35210362, concept_name='Encounter for full-term uncomplicated delivery', domain_id='Condition', CONCEPT_CODE='O80', VOCABULARY_ID='ICD10CM', CD_GRP='vaginalDelivery')

In [12]:
delivery_init = spark.sql('''
    select obs.person_id,
           obs.visit_occurrence_id, 
           obs.observation_date as event_date, 
           m.CD_GRP as delivery_type,
           m.vocabulary_id as event_source
    from observation obs
    join delivery_omop_meta m
    on obs.observation_concept_id = m.concept_id
    where upper(m.vocabulary_id) in ('DRG') 
    union all
    select px.person_id,
           px.visit_occurrence_id, 
           px.procedure_date as event_date,
           m.CD_GRP as delivery_type,
           m.vocabulary_id as event_source
    from procedure_occurrence px
    join delivery_omop_meta m
    on px.procedure_concept_id = m.concept_id
    where upper(m.vocabulary_id) in ('CPT4','HCPCS','ICD9PROC','ICD10PCS')
    union all
    select dx.person_id,
           dx.visit_occurrence_id, 
           dx.condition_start_date as event_date,
           m.CD_GRP as delivery_type,
           m.vocabulary_id as event_source
    from condition_occurrence dx
    join delivery_omop_meta m
    on dx.condition_concept_id = m.concept_id
    where upper(m.vocabulary_id) in ('ICD9CM','ICD10CM')
''').cache()
# delivery_init.createOrReplaceTempView("delivery_init")
delivery_init.write.mode('overwrite').saveAsTable("delivery_init")
delivery_init.first()

Row(person_id=25771511386, visit_occurrence_id=2345052548117, event_date=datetime.date(2021, 10, 18), delivery_type='cSection', event_source='ICD10PCS')

In [13]:
spark.sql('''
    select event_source, count(distinct person_id)
    from delivery_init
    group by event_source 
''').toPandas()

,event_source,count(DISTINCT person_id)
0,CPT4,368107
1,ICD10PCS,1341807


In [14]:
spark.sql('''
    select delivery_type, count(distinct person_id)
    from delivery_init
    group by delivery_type 
''').toPandas()

,delivery_type,count(DISTINCT person_id)
0,vaginalDelivery,972509
1,cSection,592470


In [15]:
delivery_ip = spark.sql('''
    select distinct
           a.person_id, 
           a.visit_occurrence_id,
           v.visit_start_date,
           v.visit_end_date,
           v.care_site_id
    from delivery_init a 
    join visit_occurrence v 
    on a.person_id = v.person_id and 
       a.visit_occurrence_id = v.visit_occurrence_id
    where v.visit_concept_id in (
            9201, -- IP
            9203 -- ED
         ) or 
         v.visit_source_value in (
        'I|2.16.840.1.113883.12.4|Inpatient',
        'P|2.16.840.1.113883.12.4|Preadmit',
        'E|2.16.840.1.113883.12.4|Emergency',
        'B|2.16.840.1.113883.12.4|Obstetrics'
        )
''').cache()
delivery_ip.createOrReplaceTempView("delivery_ip")
delivery_ip.first()

Row(person_id=2277, visit_occurrence_id=3298535273780, visit_start_date=datetime.date(2022, 3, 8), visit_end_date=datetime.date(2022, 3, 9), care_site_id=100)

In [16]:
delivery_consolidate = spark.sql('''
    with cd_filter as (
        select v.person_id, 
               v.visit_occurrence_id,
               v.care_site_id,
               v.visit_start_date,
               v.visit_end_date,
               a.delivery_type,
               a.event_source,
               a.event_date,
               row_number() over (partition by v.person_id, v.visit_occurrence_id, a.event_source order by a.event_date) as rn_asc,
               row_number() over (partition by v.person_id, v.visit_occurrence_id, a.event_source order by a.event_date desc) as rn_desc
        from delivery_ip v
        join delivery_init a 
        on v.person_id = a.person_id and 
           v.visit_occurrence_id = a.visit_occurrence_id
        where a.event_date between date_sub(v.visit_start_date,3) and date_add(v.visit_end_date,3)
    ), f_pvt as (
        select * 
        from (
            select person_id, visit_occurrence_id,
                   event_source, event_date
            from cd_filter
            where rn_asc = 1       
        )
        pivot (
            min(event_date) for event_source in (
                'DRG' as F_DRG_DT,'CPT4' as F_CPT_DT,'ICD10PCS' as F_ICD_DT
            )
        )
    ), l_pvt as (
        select * 
        from (
            select person_id, visit_occurrence_id,
                   event_source, event_date
            from cd_filter
            where rn_desc = 1       
        )
        pivot (
            max(event_date) for event_source in (
                'DRG' as L_DRG_DT,'CPT4' as L_CPT_DT,'ICD10PCS' as L_ICD_DT
            )
        )
    ), dtype_pvt as (
        select * 
        from (
            select person_id, visit_occurrence_id,
                   event_source, delivery_type
            from cd_filter
            where rn_desc = 1 
        )
        pivot (
            max(delivery_type) for event_source in (
                'DRG' as DTYPE_DRG,'CPT4' as DTYPE_CPT,'ICD10PCS' as DTYPE_ICD
            )
        )
    )
    select a.person_id, 
           a.visit_occurrence_id,
           a.visit_start_date,
           a.visit_end_date,
           a.care_site_id,
           f.F_DRG_DT,l.L_DRG_DT,d.DTYPE_DRG,
           f.F_CPT_DT,l.L_CPT_DT,d.DTYPE_CPT,
           f.F_ICD_DT,l.L_ICD_DT,d.DTYPE_ICD
    from delivery_ip a 
    left join f_pvt f on a.person_id = f.person_id and a.visit_occurrence_id = f.visit_occurrence_id
    left join l_pvt l on a.person_id = l.person_id and a.visit_occurrence_id = l.visit_occurrence_id
    left join dtype_pvt d on a.person_id = d.person_id and a.visit_occurrence_id = d.visit_occurrence_id
''').cache()
# delivery_consolidate.createOrReplaceTempView("delivery_consolidate")
delivery_consolidate.write.mode('overwrite').saveAsTable("delivery_consolidate")
delivery_consolidate.first()

Row(person_id=2277, visit_occurrence_id=3298535273780, visit_start_date=datetime.date(2022, 3, 8), visit_end_date=datetime.date(2022, 3, 9), care_site_id=100, F_DRG_DT=None, L_DRG_DT=None, DTYPE_DRG=None, F_CPT_DT=None, L_CPT_DT=None, DTYPE_CPT=None, F_ICD_DT=datetime.date(2022, 3, 8), L_ICD_DT=datetime.date(2022, 3, 8), DTYPE_ICD='vaginalDelivery')

In [23]:
delivery_elig = spark.sql('''
    with date_consolid as (
        select distinct 
               person_id,
               visit_occurrence_id,
               care_site_id,
               coalesce(DTYPE_DRG,DTYPE_ICD,DTYPE_CPT) as delivery_type,
               coalesce(F_DRG_DT,visit_start_date,F_ICD_DT,F_CPT_DT) as event_start_dt,
               coalesce(L_DRG_DT,visit_end_date,L_ICD_DT,L_CPT_DT) as event_end_dt
        from delivery_consolidate    
    ), visit_diffs as (
        select a.*, 
               lag(a.event_start_dt, 1, '1899-12-31') OVER (PARTITION BY person_id ORDER BY event_start_dt) AS last_event_start_dt
        from date_consolid a 
    ), visit_session as (
        select b.*, 
               case 
                   when datediff(b.event_start_dt,b.last_event_start_dt) > 211 then 1
                   else 0 
               end as new_session_flag
        from visit_diffs b
    ), sessions as (
        select d.*, 
               sum(d.new_session_flag) over (PARTITION BY d.person_id ORDER BY d.event_start_dt) as event_id
        from visit_session d
    ), session_order as (
        select e.*, 
               row_number() over (partition by e.person_id, e.event_id order by e.event_start_dt) as rn,
               max(e.event_end_dt) over (partition by e.person_id, e.event_id) as event_end_date
    from sessions e
    )
    select s.person_id, 
           s.event_id, 
           s.visit_occurrence_id,
           s.care_site_id,
           cs.care_site_source_value,
           cs.location_id,
           -- loc.state,
           -- loc.city,
           loc.zip,
           -- loc.location_source_value,
           s.delivery_type,
           s.event_start_dt as event_start_date,
           s.event_end_date
    from session_order s 
    join care_site cs on s.care_site_id = cs.care_site_id
    left join location loc on cs.location_id = loc.location_id
    where s.rn = 1
    order by s.person_id, s.event_id
''').cache()
# delivery_elig.createOrReplaceTempView("delivery_elig")
delivery_elig.write.mode('overwrite').saveAsTable("delivery_elig")
delivery_elig.first()

Row(person_id=2, event_id=1, visit_occurrence_id=2396592223081, care_site_id=100, care_site_source_value='1', location_id=8589934593, zip='8', delivery_type='cSection', event_start_date=datetime.date(2019, 5, 2), event_end_date=datetime.date(2019, 5, 5))

In [18]:
spark.sql('''
    select event_id, count(distinct person_id)
    from delivery_elig
    group by event_id
    order by event_id
''').toPandas()

,event_id,count(DISTINCT person_id)
0,1,1419426
1,2,323090
2,3,51359
3,4,6901
4,5,858
5,6,123
6,7,14
7,8,1


In [36]:
delivery_elig_tbl1 = spark.sql('''
    select d.person_id,
           d.event_id,
           d.event_start_date, 
           d.event_end_date,
           d.delivery_type,
           coalesce(datediff(d.event_end_date,d.event_start_date),0) as los, 
           d.visit_occurrence_id,
           p.year_of_birth,
           year(d.event_start_date) - p.year_of_birth as age_at_event,
           --p.month_of_birth,
           --p.day_of_birth,
           p.race_source_value,
           p.ethnicity_source_value,
           p.location_id as pt_location_id,
           ploc.zip as pt_zip,
           p.care_site_id as pcp_care_site_id,
           loc.zip as pcp_care_site_zip,
           d.care_site_id,
           d.location_id,
           d.zip,
           case when d.zip in (0,1) then 'NE'
                when d.zip in (2,3,7) then 'S'
                when d.zip in (4,5,6) then 'MW'
                when d.zip in (8,9) then 'W'
                else 'NI'
           end as zip_code_region, 
           coalesce(tnt.bed_size,'NI') as bed_size,
           tnt.speciality,
           tnt.segment,
           dth.death_date,
           case when dth.death_date is not null then 1 else 0 end as death_ind,
           row_number() over (partition by d.person_id order by d.event_start_date) as delivery_idx
    from delivery_elig d
    join person p on d.person_id = p.person_id
    left join location ploc on p.location_id = ploc.location_id
    left join care_site cs on cs.care_site_id = p.care_site_id
    left join location loc on cs.location_id = loc.location_id
    left join tenant_attributes tnt on d.care_site_source_value = tnt.tenant
    left join death dth on d.person_id = dth.person_id 
    where year(d.event_start_date) - p.year_of_birth between 10 and 55 and 
          event_start_date between '2015-01-01' and '2024-12-31'
''').cache()
# delivery_elig_tbl1.createOrReplaceTempView("delivery_elig_tbl1")
delivery_elig_tbl1.write.mode('overwrite').saveAsTable("delivery_elig_tbl1")
delivery_elig_tbl1.first()

Row(person_id=14719, event_id=1, event_start_date=datetime.date(2024, 5, 31), event_end_date=datetime.date(2024, 6, 3), delivery_type='vaginalDelivery', los=3, visit_occurrence_id=3066607144971, year_of_birth=1992, age_at_event=32, race_source_value='American Indian or Alaska Native', ethnicity_source_value='Non-Hispanic', pt_location_id=23, pt_zip=None, pcp_care_site_id=100, pcp_care_site_zip='8', care_site_id=100, location_id=8589934593, zip='8', zip_code_region='W', bed_size='200-299', speciality='Acute Care Hospital', segment='Hospital', death_date=None, death_ind=0, delivery_idx=1)

In [37]:
spark.sql('''
    select count(distinct person_id), count(*)
    from delivery_elig_tbl1
    where delivery_idx = 1
''').toPandas()

,count(DISTINCT person_id),count(1)
0,1385517,1385517


In [46]:
spark.sql('''
    select 'all' as region, count(distinct care_site_id), count(distinct location_id), count(distinct person_id), count(distinct pt_location_id)
    from delivery_elig_tbl1
    union 
    select zip_code_region, count(distinct care_site_id), count(distinct location_id), count(distinct person_id), count(distinct pt_location_id)
    from delivery_elig_tbl1 
    group by zip_code_region 
''').toPandas()

,region,count(DISTINCT care_site_id),count(DISTINCT location_id),count(DISTINCT person_id),count(DISTINCT pt_location_id)
0,MW,58,3,241179,52
1,NE,18,2,250195,52
2,S,23,3,340385,52
3,all,122,10,1385517,52
4,W,23,2,553758,52


In [47]:
# load SMM code list and get omop concept_id
vs_smm = QueryFromJson(
    url = 'https://raw.githubusercontent.com/RWD2E/phecdm/refs/heads/main/res/valueset_curated/vs-mmm-cde.json',
    sqlty = 'spark',
    cd_field = 'CONCEPT_CODE',
    cdtype_field = 'VOCABULARY_ID',
    srctbl_name = "CONCEPT",
    other_fields=["concept_id","concept_name","domain_id"],
    sel_keys = [
        'ami',
        'ane',
        'arf',
        'ards',
        'afe',
        'cavf',
        'cocr',
        'dic',
        'ecl',
        'hf',
        'pcd',
        'pe',
        'sac',
        'sep',
        'ssh',
        'scc',
        'ate',
        'bpt',
        'hys',
        'tt',
        'ven'
    ]
)
vs_smm_qry = vs_smm.gen_qry()
print(vs_smm_qry)

Enter Code Type Value for icd9cm: ICD9CM
Enter Code Type Value for icd10cm: ICD10CM
Enter Code Type Value for icd9proc: ICD9PROC
Enter Code Type Value for icd10pcs: ICD10PCS
Enter Code Type Value for cpt: CPT4
Enter Code Type Value for hpc: HCPCS
Enter Code Type Value for drg: MSDRG
Enter Code Type Value for loinc: LOINC

                    select concept_id,concept_name,domain_id,CONCEPT_CODE,VOCABULARY_ID ,'ami' as CD_GRP
                    from CONCEPT
                    where (VOCABULARY_ID='ICD9CM' and (
            substring_index(CONCEPT_CODE,'.',1)
         in ('410')
                        )) OR (VOCABULARY_ID='ICD10CM' and (
            substring_index(CONCEPT_CODE,'.',1)
         in ('I21','I22')
                        ))
                 union all 
                    select concept_id,concept_name,domain_id,CONCEPT_CODE,VOCABULARY_ID ,'ane' as CD_GRP
                    from CONCEPT
                    where (VOCABULARY_ID='ICD9CM' and (
            substring_index(CO

In [48]:
smm_omop_meta = spark.sql(vs_smm_qry).cache()
smm_omop_meta.createOrReplaceTempView("smm_omop_meta")
smm_omop_meta.first()

Row(concept_id=1569126, concept_name='Acute myocardial infarction', domain_id='Condition', CONCEPT_CODE='I21', VOCABULARY_ID='ICD10CM', CD_GRP='ami')

In [49]:
smm_init = spark.sql('''
    select px.person_id,
           px.visit_occurrence_id, 
           px.procedure_date as event_date,
           m.CD_GRP as SMM_GRP
    from procedure_occurrence px
    join smm_omop_meta m
    on px.procedure_concept_id = m.concept_id
    where upper(m.vocabulary_id) in ('CPT4','HCPCS','ICD9PROC','ICD10PCS')
    union all
    select person_id,
           visit_occurrence_id, 
           condition_start_date as event_date,
           m.CD_GRP as SMM_GRP
    from condition_occurrence dx
    join smm_omop_meta m
    on dx.condition_concept_id = m.concept_id
    where upper(m.vocabulary_id) in ('ICD9CM','ICD10CM')
''').cache()
smm_init.createOrReplaceTempView("smm_init")
smm_init.first()

Row(person_id=274879532846, visit_occurrence_id=2997887245681, event_date=datetime.date(2019, 4, 19), SMM_GRP='bpt')

In [50]:
smm_post_delivery = spark.sql('''
    select a.person_id, b.event_id,
           a.SMM_GRP,
           b.event_start_date,a.event_date,
           datediff(a.event_date,b.event_start_date) AS days_since_index,
           b.delivery_idx
    from smm_init a 
    join delivery_elig_tbl1 b 
    on a.person_id = b.person_id
    where datediff(a.event_date,b.event_start_date) between 0 and 365
''').cache()
smm_post_delivery.createOrReplaceTempView("smm_post_delivery")
smm_post_delivery.first()

Row(person_id=8591565214, event_id=1, SMM_GRP='bpt', event_start_date=datetime.date(2023, 3, 31), event_date=datetime.date(2023, 4, 3), days_since_index=3, delivery_idx=1)

In [51]:
smm_post_delivery_wide = spark.sql('''
    select *
    from (
        select person_id, event_id, SMM_GRP,days_since_index
        from smm_post_delivery
     )
    pivot 
    (
        min(days_since_index) for SMM_GRP in (
            'ami' as AMI_since_index,
            'ane' as ANE_since_index,
            'arf' as ARF_since_index,
            'ards' as ARDS_since_index,
            'afe' as AFE_since_index,
            'cavf' as CAVF_since_index,
            'cocr' as COCR_since_index,
            'dic' as DIC_since_index,
            'ecl' as ECL_since_index,
            'hf' as HF_since_index,
            'pcd' as PCD_since_index,
            'pe' as PE_since_index,
            'sac' as SAC_since_index,
            'sep' as SEP_since_index,
            'ssh' as SSH_since_index,
            'scc' as SCC_since_index,
            'ate' as ATE_since_index,
            'bpt' as BPT_since_index,
            'hys' as HYS_since_index,
            'tt' as TT_since_index,
            'ven' as VEN_since_index
        )
    )
''').cache()
# smm_post_delivery_wide.createOrReplaceTempView("smm_post_delivery_wide")
smm_post_delivery_wide.write.mode('overwrite').saveAsTable("smm_post_delivery_wide")
smm_post_delivery_wide.first()

Row(person_id=77309970066, event_id=2, AMI_since_index=None, ANE_since_index=None, ARF_since_index=None, ARDS_since_index=None, AFE_since_index=None, CAVF_since_index=None, COCR_since_index=None, DIC_since_index=None, ECL_since_index=None, HF_since_index=None, PCD_since_index=None, PE_since_index=None, SAC_since_index=None, SEP_since_index=None, SSH_since_index=None, SCC_since_index=None, ATE_since_index=None, BPT_since_index=1, HYS_since_index=None, TT_since_index=None, VEN_since_index=None)

In [52]:
delivery_elig_censor = spark.sql('''
    select a.person_id, a.event_id,
           max(coalesce(a.death_date,v.visit_end_date,v.visit_start_date)) as censor_date,
           datediff(max(coalesce(a.death_date,v.visit_end_date,v.visit_start_date)),a.event_start_date) as censor_since_index
    from delivery_elig_tbl1 a 
    join visit_occurrence v
    on a.person_id = v.person_id
    group by a.person_id,a.event_id,a.event_start_date
''').cache()
# delivery_elig_censor.createOrReplaceTempView("delivery_elig_censor")
delivery_elig_censor.write.mode('overwrite').saveAsTable("delivery_elig_censor")
delivery_elig_censor.first()

Row(person_id=214749853091, event_id=1, censor_date=datetime.date(2024, 11, 11), censor_since_index=2231)

In [53]:
# load comorb code list and get omop concept_id
vs_obcmi = QueryFromJson(
    url = 'https://raw.githubusercontent.com/RWD2E/phecdm/refs/heads/main/res/valueset_curated/vs-comorb-OBCMI.json',
    sqlty = 'spark',
    cd_field = 'CONCEPT_CODE',
    cdtype_field = 'VOCABULARY_ID',
    srctbl_name = "CONCEPT",
    other_fields=["concept_id","concept_name","domain_id"]
)
vs_obcmi_qry = vs_obcmi.gen_qry()
print(vs_obcmi_qry)

Enter Code Type Value for icd9cm: ICD9CM
Enter Code Type Value for icd10cm: ICD10CM
Enter Code Type Value for icd9proc: ICD9PROC
Enter Code Type Value for icd10pcs: ICD10PCS
Enter Code Type Value for cpt: CPT4
Enter Code Type Value for hpc: HCPCS
Enter Code Type Value for drg: MSDRG
Enter Code Type Value for loinc: LOINC

                    select concept_id,concept_name,domain_id,CONCEPT_CODE,VOCABULARY_ID ,'PHT' as CD_GRP
                    from CONCEPT
                    where (VOCABULARY_ID='ICD9CM' and (
                        substring(CONCEPT_CODE,1,5) in ('416.0','416.8','416.9')
                    )) OR (VOCABULARY_ID='ICD10CM' and (
                        substring(CONCEPT_CODE,1,5) in ('I27.0','I27.2')
                    ))
                 union all 
                    select concept_id,concept_name,domain_id,CONCEPT_CODE,VOCABULARY_ID ,'PEC' as CD_GRP
                    from CONCEPT
                    where (VOCABULARY_ID='ICD9CM' and (
                        su

In [54]:
comorb_omop_meta = spark.sql(vs_obcmi_qry).cache()
comorb_omop_meta.createOrReplaceTempView("comorb_omop_meta")
comorb_omop_meta.first()

Row(concept_id=35207707, concept_name='Primary pulmonary hypertension', domain_id='Condition', CONCEPT_CODE='I27.0', VOCABULARY_ID='ICD10CM', CD_GRP='PHT')

In [55]:
comorb_init = spark.sql('''
    select person_id,
           visit_occurrence_id, 
           condition_start_date as event_date,
           m.CD_GRP as COMORB_GRP
    from condition_occurrence dx
    join comorb_omop_meta m
    on dx.condition_source_concept_id = m.concept_id
    where upper(m.vocabulary_id) in ('ICD9CM','ICD10CM')
''').cache()
comorb_init.createOrReplaceTempView("comorb_init")
comorb_init.first()

Row(person_id=214750189671, visit_occurrence_id=10058813535961, event_date=datetime.date(2023, 9, 18), COMORB_GRP='CHF')

In [56]:
comorb_pre_delivery = spark.sql('''
    select a.person_id, b.event_id,
           a.COMORB_GRP,
           b.event_start_date,a.event_date,
           datediff(a.event_date,b.event_start_date) AS days_since_index,
           b.delivery_idx
    from comorb_init a 
    join delivery_elig_tbl1 b 
    on a.person_id = b.person_id
    where datediff(a.event_date,b.event_start_date) < 0
''').cache()
comorb_pre_delivery.createOrReplaceTempView("comorb_pre_delivery")
comorb_pre_delivery.first()

Row(person_id=171799213770, event_id=2, COMORB_GRP='HistPD', event_start_date=datetime.date(2024, 9, 25), event_date=datetime.date(2023, 5, 22), days_since_index=-492, delivery_idx=2)

In [57]:
comorb_pre_delivery_wide = spark.sql('''
    select *
    from (
        select person_id, event_id, COMORB_GRP,days_since_index
        from comorb_pre_delivery
     )
    pivot 
    (
        min(days_since_index) for COMORB_GRP in (
            'PHT' as hist_PHT_since_index,
            'PEC' as hist_PEC_since_index,
            'GHT' as hist_GHT_since_index,
            'PAS' as hist_PAS_since_index,
            'CKD' as hist_CKD_since_index,
            'HistHD' as hist_HD_since_index,
            'HIV' as hist_HIV_since_index,
            'PA' as hist_PA_since_index,
            'HistCD' as hist_CD_since_index,
            'HistANEM' as hist_ANEM_since_index,
            'MULTIPG' as hist_MULTIPG_since_index,
            'PP' as hist_PP_since_index,
            'NMD' as hist_NMD_since_index,
            'ASTH' as hist_ASTH_since_index,
            'AID' as hist_AID_since_index,
            'UF' as hist_UF_since_index,
            'GID' as hist_GID_since_index,
            'CH' as hist_CH_since_index,
            'HistPD' as hist_PD_since_index,
            'HistDM' as hist_DM_since_index,
            'THYRO' as hist_THYRO_since_index,
            'HistCSEC' as hist_CSEC_since_index,
            'GDM' as hist_GDM_since_index,
            'IHD' as hist_IHD_since_index,
            'ARRY' as hist_ARRY_since_index,
            'CHF' as hist_CHF_since_index,
            'CHD' as hist_CHD_since_index,
            'RA' as hist_RA_since_index,
            'PVD' as hist_PVD_since_index,
            'EPI' as hist_EPI_since_index,
            'CVD' as hist_CVD_since_index,
            'SCD' as hist_SCD_since_index,
            'SUD' as hist_SUD_since_index,
            'AUD' as hist_AUD_since_index,
            'COVID' as hist_COVID_since_index
        )
    )
''').cache()
# comorb_pre_delivery_wide.createOrReplaceTempView("comorb_pre_delivery_wide")
comorb_pre_delivery_wide.write.mode('overwrite').saveAsTable("comorb_pre_delivery_wide")
comorb_pre_delivery_wide.first()

Row(person_id=395138767902, event_id=1, hist_PHT_since_index=None, hist_PEC_since_index=None, hist_GHT_since_index=None, hist_PAS_since_index=None, hist_CKD_since_index=None, hist_HD_since_index=None, hist_HIV_since_index=None, hist_PA_since_index=None, hist_CD_since_index=None, hist_ANEM_since_index=None, hist_MULTIPG_since_index=None, hist_PP_since_index=None, hist_NMD_since_index=None, hist_ASTH_since_index=-95, hist_AID_since_index=None, hist_UF_since_index=None, hist_GID_since_index=None, hist_CH_since_index=None, hist_PD_since_index=None, hist_DM_since_index=None, hist_THYRO_since_index=None, hist_CSEC_since_index=None, hist_GDM_since_index=None, hist_IHD_since_index=None, hist_ARRY_since_index=None, hist_CHF_since_index=None, hist_CHD_since_index=None, hist_RA_since_index=None, hist_PVD_since_index=None, hist_EPI_since_index=None, hist_CVD_since_index=None, hist_SCD_since_index=None, hist_SUD_since_index=None, hist_AUD_since_index=-96, hist_COVID_since_index=None)

In [18]:
delivery_elig_smm = spark.sql('''
    with smm_any as (
        select person_id, event_id, 1 as SMMANY_ind,
               min(days_since_index) as SMMANY_since_index
        from smm_post_delivery
        group by person_id, event_id
    )
    select e.*,
           case when e.age_at_event >=10 and e.age_at_event <20 then 'agegrp1'
                when e.age_at_event >=20 and e.age_at_event <25 then 'agegrp2'
                when e.age_at_event >=25 and e.age_at_event <30 then 'agegrp3'
                when e.age_at_event >=30 and e.age_at_event <35 then 'agegrp4'
                when e.age_at_event >=35 and e.age_at_event <40 then 'agegrp5'
                else 'agegrp6' 
           end as agegrp_at_event,
           case when los >= 2 then 1 else 0 end as los2up_ind,
           case when los >= 3 then 1 else 0 end as los3up_ind,
           case when los >= 4 then 1 else 0 end as los4up_ind,
           case when los >= 5 then 1 else 0 end as los5up_ind,
           case when los >= 6 then 1 else 0 end as los6up_ind,
           case when los >= 7 then 1 else 0 end as los7up_ind,
           cs.censor_since_index,
           coalesce(datediff(e.death_date,e.event_start_date),cs.censor_since_index) as death_since_index,
           coalesce(a.SMMANY_ind,0) as SMMANY_ind,
           coalesce(a.SMMANY_since_index,cs.censor_since_index) as SMMANY_since_index,
           coalesce(s.AMI_since_index,cs.censor_since_index) as AMI_since_index,
           IF(s.AMI_since_index IS NOT NULL, 1, 0) AMI_ind,
           coalesce(s.ANE_since_index,cs.censor_since_index) as ANE_since_index,
           IF(s.ANE_since_index IS NOT NULL, 1, 0) ANE_ind,
           coalesce(s.ARF_since_index,cs.censor_since_index) as ARF_since_index,
           IF(s.ARF_since_index IS NOT NULL, 1, 0) ARF_ind,
           coalesce(s.ARDS_since_index,cs.censor_since_index) as ARDS_since_index,
           IF(s.ARDS_since_index IS NOT NULL, 1, 0) ARDS_ind,
           coalesce(s.AFE_since_index,cs.censor_since_index) as AFE_since_index,
           IF(s.AFE_since_index IS NOT NULL, 1, 0) AFE_ind,
           coalesce(s.CAVF_since_index,cs.censor_since_index) as CAVF_since_index,
           IF(s.CAVF_since_index IS NOT NULL, 1, 0) CAVF_ind,
           coalesce(s.COCR_since_index,cs.censor_since_index) as COCR_since_index,
           IF(s.COCR_since_index IS NOT NULL, 1, 0) COCR_ind,
           coalesce(s.DIC_since_index,cs.censor_since_index) as DIC_since_index,
           IF(s.DIC_since_index IS NOT NULL, 1, 0) DIC_ind,
           coalesce(s.ECL_since_index,cs.censor_since_index) as ECL_since_index,
           IF(s.ECL_since_index IS NOT NULL, 1, 0) ECL_ind,
           coalesce(s.HF_since_index,cs.censor_since_index) as HF_since_index,
           IF(s.HF_since_index IS NOT NULL, 1, 0) HF_ind,
           coalesce(s.PCD_since_index,cs.censor_since_index) as PCD_since_index,
           IF(s.PCD_since_index IS NOT NULL, 1, 0) PCD_ind,
           coalesce(s.PE_since_index,cs.censor_since_index) as PE_since_index,
           IF(s.PE_since_index IS NOT NULL, 1, 0) PE_ind,
           coalesce(s.SAC_since_index,cs.censor_since_index) as SAC_since_index,
           IF(s.SAC_since_index IS NOT NULL, 1, 0) SAC_ind,
           coalesce(s.SEP_since_index,cs.censor_since_index) as SEP_since_index,
           IF(s.SEP_since_index IS NOT NULL, 1, 0) SEP_ind,
           coalesce(s.SSH_since_index,cs.censor_since_index) as SSH_since_index,
           IF(s.SSH_since_index IS NOT NULL, 1, 0) SSH_ind,
           coalesce(s.SCC_since_index,cs.censor_since_index) as SCC_since_index,
           IF(s.SCC_since_index IS NOT NULL, 1, 0) SCC_ind,
           coalesce(s.ATE_since_index,cs.censor_since_index) as ATE_since_index,
           IF(s.ATE_since_index IS NOT NULL, 1, 0) ATE_ind,
           coalesce(s.BPT_since_index,cs.censor_since_index) as BPT_since_index,
           IF(s.BPT_since_index IS NOT NULL, 1, 0) BPT_ind,
           coalesce(s.HYS_since_index,cs.censor_since_index) as HYS_since_index,
           IF(s.HYS_since_index IS NOT NULL, 1, 0) HYS_ind,
           coalesce(s.TT_since_index,cs.censor_since_index) as TT_since_index,
           IF(s.TT_since_index IS NOT NULL, 1, 0) TT_ind,
           coalesce(s.VEN_since_index,cs.censor_since_index) as VEN_since_index,
           IF(s.VEN_since_index IS NOT NULL, 1, 0) VEN_ind,
           cmb.hist_PHT_since_index,
           IF(cmb.hist_PHT_since_index IS NOT NULL, 1, 0) hist_PHT_ind,
           cmb.hist_PEC_since_index,
           IF(cmb.hist_PEC_since_index IS NOT NULL, 1, 0) hist_PEC_ind,
           cmb.hist_GHT_since_index,
           IF(cmb.hist_GHT_since_index IS NOT NULL, 1, 0) hist_GHT_ind,
           cmb.hist_PAS_since_index,
           IF(cmb.hist_PAS_since_index IS NOT NULL, 1, 0) hist_PAS_ind,
           cmb.hist_CKD_since_index,
           IF(cmb.hist_CKD_since_index IS NOT NULL, 1, 0) hist_CKD_ind,
           cmb.hist_HD_since_index,
           IF(cmb.hist_HD_since_index IS NOT NULL, 1, 0) hist_HD_ind,
           cmb.hist_HIV_since_index,
           IF(cmb.hist_HIV_since_index IS NOT NULL, 1, 0) hist_HIV_ind,
           cmb.hist_PA_since_index,
           IF(cmb.hist_PA_since_index IS NOT NULL, 1, 0) hist_PA_ind,
           cmb.hist_CD_since_index,
           IF(cmb.hist_CD_since_index IS NOT NULL, 1, 0) hist_CD_ind,
           cmb.hist_ANEM_since_index,
           IF(cmb.hist_ANEM_since_index IS NOT NULL, 1, 0) hist_ANEM_ind,
           cmb.hist_MULTIPG_since_index,
           IF(cmb.hist_MULTIPG_since_index IS NOT NULL, 1, 0) hist_MULTIPG_ind,
           cmb.hist_PP_since_index,
           IF(cmb.hist_PP_since_index IS NOT NULL, 1, 0) hist_PP_ind,
           cmb.hist_NMD_since_index,
           IF(cmb.hist_NMD_since_index IS NOT NULL, 1, 0) hist_NMD_ind,
           cmb.hist_ASTH_since_index,
           IF(cmb.hist_ASTH_since_index IS NOT NULL, 1, 0) hist_ASTH_ind,
           cmb.hist_AID_since_index,
           IF(cmb.hist_AID_since_index IS NOT NULL, 1, 0) hist_AID_ind,
           cmb.hist_UF_since_index,
           IF(cmb.hist_UF_since_index IS NOT NULL, 1, 0) hist_UF_ind,
           cmb.hist_GID_since_index,
           IF(cmb.hist_GID_since_index IS NOT NULL, 1, 0) hist_GID_ind,
           cmb.hist_CH_since_index,
           IF(cmb.hist_CH_since_index IS NOT NULL, 1, 0) hist_CH_ind,
           cmb.hist_PD_since_index,
           IF(cmb.hist_PD_since_index IS NOT NULL, 1, 0) hist_PD_ind,
           cmb.hist_DM_since_index,
           IF(cmb.hist_DM_since_index IS NOT NULL, 1, 0) hist_DM_ind,
           cmb.hist_THYRO_since_index,
           IF(cmb.hist_THYRO_since_index IS NOT NULL, 1, 0) hist_THYRO_ind,
           cmb.hist_CSEC_since_index,
           IF(cmb.hist_CSEC_since_index IS NOT NULL, 1, 0) hist_CSEC_ind,
           cmb.hist_GDM_since_index,
           IF(cmb.hist_GDM_since_index IS NOT NULL, 1, 0) hist_GDM_ind,
           cmb.hist_IHD_since_index,
           IF(cmb.hist_IHD_since_index IS NOT NULL, 1, 0) hist_IHD_ind,
           cmb.hist_ARRY_since_index,
           IF(cmb.hist_ARRY_since_index IS NOT NULL, 1, 0) hist_ARRY_ind,
           cmb.hist_CHF_since_index,
           IF(cmb.hist_CHF_since_index IS NOT NULL, 1, 0) hist_CHF_ind,
           cmb.hist_CHD_since_index,
           IF(cmb.hist_CHD_since_index IS NOT NULL, 1, 0) hist_CHD_ind,
           cmb.hist_RA_since_index,
           IF(cmb.hist_RA_since_index IS NOT NULL, 1, 0) hist_RA_ind,
           cmb.hist_PVD_since_index,
           IF(cmb.hist_PVD_since_index IS NOT NULL, 1, 0) hist_PVD_ind,
           cmb.hist_EPI_since_index,
           IF(cmb.hist_EPI_since_index IS NOT NULL, 1, 0) hist_EPI_ind,
           cmb.hist_CVD_since_index,
           IF(cmb.hist_CVD_since_index IS NOT NULL, 1, 0) hist_CVD_ind,
           cmb.hist_SCD_since_index,
           IF(cmb.hist_SCD_since_index IS NOT NULL, 1, 0) hist_SCD_ind,
           cmb.hist_SUD_since_index,
           IF(cmb.hist_SUD_since_index IS NOT NULL, 1, 0) hist_SUD_ind,
           cmb.hist_AUD_since_index,
           IF(cmb.hist_AUD_since_index IS NOT NULL, 1, 0) hist_AUD_ind,
           cmb.hist_COVID_since_index,
           IF(cmb.hist_COVID_since_index IS NOT NULL, 1, 0) hist_COVID_ind
    from delivery_elig_tbl1 e
    left join smm_any a 
    on e.person_id = a.person_id and e.event_id = a.event_id
    left join smm_post_delivery_wide s 
    on e.person_id = s.person_id and e.event_id = s.event_id
    left join delivery_elig_censor cs
    on e.person_id = cs.person_id and e.event_id = cs.event_id
    left join comorb_pre_delivery_wide cmb 
    on e.person_id = cmb.person_id and e.event_id = cmb.event_id 
    where cs.censor_since_index > 0  
''').cache()
# delivery_elig_smm.createOrReplaceTempView("delivery_elig_smm")
delivery_elig_smm.write.mode('overwrite').saveAsTable("delivery_elig_smm")
delivery_elig_smm.first()

Row(person_id=2611, event_id=1, event_start_date=datetime.date(2020, 9, 16), event_end_date=datetime.date(2020, 9, 18), delivery_type='cSection', los=2, visit_occurrence_id=2568390892667, year_of_birth=1995, age_at_event=25, race_source_value='American Indian or Alaska Native', ethnicity_source_value='Non-Hispanic', location_id=23, care_site_id=100, care_site_source_value='1', bed_size='200-299', speciality='Acute Care Hospital', segment='Hospital', zip_code='8', zip_code_region='W', death_date=None, death_ind=0, delivery_idx=1, agegrp_at_event='agegrp3', los2up_ind=1, los3up_ind=0, los4up_ind=0, los5up_ind=0, los6up_ind=0, los7up_ind=0, censor_since_index=960, death_since_index=960, SMMANY_ind=0, SMMANY_since_index=960, AMI_since_index=960, AMI_ind=0, ANE_since_index=960, ANE_ind=0, ARF_since_index=960, ARF_ind=0, ARDS_since_index=960, ARDS_ind=0, AFE_since_index=960, AFE_ind=0, CAVF_since_index=960, CAVF_ind=0, COCR_since_index=960, COCR_ind=0, DIC_since_index=960, DIC_ind=0, ECL_sin

In [19]:
delivery_elig_init_smm = spark.sql('''
    with cte as (
        select percentile(los,0.9) as los_90pct
        from delivery_elig_smm
    )
    select a.*, 
           case when a.SMMANY_ind = 1 and a.los>=cte.los_90pct then 1 else 0 end as SMMANY90PCT_ind,
           case when a.SMMANY_ind = 1 and a.los>=cte.los_90pct then SMMANY_since_index else censor_since_index end as SMMANY90PCT_since_index
    from delivery_elig_smm a
    cross join cte
    where a.delivery_idx = 1 and 
          a.delivery_type in ('vaginalDelivery','cSection')
''').cache()
# delivery_elig_init_smm.createOrReplaaceTempView("delivery_elig_init_smm")
delivery_elig_init_smm.write.mode('overwrite').saveAsTable("delivery_elig_init_smm")
delivery_elig_init_smm.first()

Row(person_id=4473, event_id=1, event_start_date=datetime.date(2020, 8, 30), event_end_date=datetime.date(2020, 9, 1), delivery_type='vaginalDelivery', los=2, visit_occurrence_id=3109556421910, year_of_birth=1993, age_at_event=27, race_source_value='American Indian or Alaska Native', ethnicity_source_value='Non-Hispanic', location_id=23, care_site_id=100, care_site_source_value='1', bed_size='200-299', speciality='Acute Care Hospital', segment='Hospital', zip_code='8', zip_code_region='W', death_date=None, death_ind=0, delivery_idx=1, agegrp_at_event='agegrp3', los2up_ind=1, los3up_ind=0, los4up_ind=0, los5up_ind=0, los6up_ind=0, los7up_ind=0, censor_since_index=1519, death_since_index=1519, SMMANY_ind=0, SMMANY_since_index=1519, AMI_since_index=1519, AMI_ind=0, ANE_since_index=1519, ANE_ind=0, ARF_since_index=1519, ARF_ind=0, ARDS_since_index=1519, ARDS_ind=0, AFE_since_index=1519, AFE_ind=0, CAVF_since_index=1519, CAVF_ind=0, COCR_since_index=1519, COCR_ind=0, DIC_since_index=1519, D

In [20]:
spark.sql('''
    select count(distinct person_id), count(*)
    from delivery_elig_init_smm
''').toPandas()

,count(DISTINCT person_id),count(1)
0,1251013,1251013


In [21]:
spark.sql('''
    select count(distinct person_id), count(*)
    from delivery_elig_smm
''').toPandas()

,count(DISTINCT person_id),count(1)
0,1255779,1571506
